In [1]:
import scipy.stats as st
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
import testjx
import numpy as np
import cvxopt
import cv2
import os 
import random
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from cvxopt import matrix
from cvxopt import solvers
import face_recognition
import sklearn.metrics as sm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from scikitplot import plotters as skplt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from statsmodels.discrete.discrete_model import Logit, Probit, MNLogit
from pylab import mpl

# import warning
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib as mpl
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from collections import Counter
import cv2
import os
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
import preprocessing
from tqdm import tqdm
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter
import imblearn
from sklearn.metrics import balanced_accuracy_score
sn.set()
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC # SVC
from sklearn import metrics
from sklearn.utils import shuffle
from xgboost import XGBClassifier # XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score,recall_score,cohen_kappa_score,precision_score
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier # RandomForestClassifier
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.nasnet import NASNetMobile # NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Feature extraction

In [2]:
''' Feature extraction---VGG19 network'''
train=[]
for i in range(16207):
# for i in range(20000):
    img_ = cv2.imread('/Users/lvjingzhe/Downloads/celebA/trainset_bangs/%d.jpg' %(i))

    train.append(img_)
train_image=np.array(train)
import time
start=time.time()
base_model= VGG19( weights='imagenet', include_top=False,input_shape=(64,64,3))
x = base_model.output
# x = Dropout(0.2)(x)
predictions = Flatten()(x)

model_feat = Model(inputs=base_model.input,outputs=predictions)

train_features = model_feat.predict(train_image/255)
# test_features=model_feat.predict(test_image/255)
end=time.time()
print('running time:{}'.format(end-start))

W0517 11:33:02.768800 4589504000 deprecation.py:506] From /Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


running time:248.9957413673401


In [4]:
'''Divide into two sets: training set(x_tr,y_tr), and testing set (x_te,y_te)'''

train_label=np.concatenate((np.array([0]*13766),np.array([1]*2441)))


x_tr, x_te, y_tr, y_te = train_test_split(train_features,train_label,test_size = 0.8,
                                                  shuffle = True,
                                                  random_state = 42)

In [5]:
E=np.hstack((train_features,train_label.reshape(16207,1)))
E.shape

(16207, 2049)

In [6]:
'''Shuffle the dataset'''
np.random.seed(4123)
np.random.shuffle(E)
X_sample=E[:,:-1]
y_sample=E[:,-1]
X_sample.shape

(16207, 2048)

In [7]:
'''Data preprocessing'''

X_sample_XX=np.c_[X_sample,np.ones(X_sample.shape[0])]

In [8]:
'''Data preprocessing'''
X_train=np.c_[x_tr,np.ones(x_tr.shape[0])]
X_test=np.c_[x_te,np.ones(x_te.shape[0])]

In [9]:
'''Imbalanced ratio'''
train_label.shape[0]/sum(train_label)

6.6394920114707086

In [10]:
'''Split the training set into 20 groups, one of which contains the toral glassed ones.
This step is useful in max-mean loss method
 (x_tr,y_tr) becomes (pre_XX,pre_YY) after "Split", which is the input of max-mean loss method.'''
def split(x,y):
    k=int(y.shape[0]/sum(y))-1
    n=y[y==1].shape[0]
    m=int((x.shape[0]-n)/k)
    indice=[i*m for i in range(1,k)]
    indice.append(x.shape[0]-n)
    x_c=np.concatenate((x[y==0],x[y==1]))
    y_c=np.concatenate((np.array([0]*(x.shape[0]-n)),np.array([1]*n)))
    return np.array(np.split(x_c,indice)),np.array(np.split(y_c,indice))


n=y_tr[y_tr==1].shape[0]
XX=np.concatenate((X_train[y_tr==0],X_train[y_tr==1]))
YY=np.concatenate((np.array([0]*(X_train.shape[0]-n)),np.array([1]*n)))
m=int((X_train.shape[0]-n)/5)
indice=[k*m for k in range(1,5)]
indice.append(X_train.shape[0]-n)

pre_XX=np.array(np.split(XX,indice))
pre_YY=np.array(np.split(YY,indice))

In [11]:
''' Split training set into 5 cross-validation sets'''
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=0)

## Traditional classifiers：LR、CS_LR、AUSTBOOST、BAGGING

In [41]:
'''model_in symbols the result of traditional logistic regression'''

import time
start_i=time.time()
model_in=LogisticRegression()

model_in.fit(x_tr,y_tr)
end_i=time.time()
y_train_proba=model_in.predict_proba(x_tr)
y_train_label=model_in.predict(x_tr)
y_test_proba=model_in.predict_proba(x_te)

y_test_label=model_in.predict(x_te)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end_i-start_i))
from sklearn.metrics import accuracy_score

print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
# print(classification_report(y_tr,y_train_label))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95     10998
           1       0.76      0.69      0.72      1968

    accuracy                           0.92     12966
   macro avg       0.85      0.83      0.84     12966
weighted avg       0.92      0.92      0.92     12966

Running time:0.97 s
Balanced-Accuracy on testing set：82.64%
Recall on testing set：69.16%
F-measure on testing set：69.82%


In [34]:
'''Return the baseline classifiers'''
from sklearn import tree
def get_models():
	models, names = list(), list()
	# LR
	models.append(LogisticRegression())
	names.append('LR')
	# SVM
	models.append(SVC(probability=True))
	names.append('SVM')
	# Bagging
	models.append(BaggingClassifier())
	names.append('BAG')
	models.append(AdaBoostClassifier(random_state=0))
	names.append('Adaboost')
	# RF
	models.append(
	tree.DecisionTreeClassifier(max_leaf_nodes=6,
                               random_state=0))
	names.append('CART')
	models.append(RandomForestClassifier(criterion='entropy',n_estimators=6))
	names.append('RF')
	# GBM
	models.append(MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,)))
    
	names.append('MLP')

	return models, names

import time

'''Out put the average performances of the classifier'''
def fivetrials(X,y,model,sample='none'):
    pipeline = model
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    if sample!='none':

        for train_index,test_index in cv.split(X, y):
        
            start=time.time()
            resample_x,resample_y=sample.fit_resample(X[test_index],y[test_index])
            
            pipeline.fit(resample_x,resample_y)
            end=time.time()
            prdict_y=pipeline.predict(X[train_index])
            prob_y=pipeline.predict_proba(X[train_index])
            rec.append(recall_score(y[train_index],prdict_y))
            G_m.append(np.sqrt(recall_score(y[train_index],prdict_y)*recall_score(y[train_index],prdict_y,pos_label=0)))
            pre.append(precision_score(y[train_index],prdict_y))
            bacc.append(balanced_accuracy_score(y[train_index],prdict_y))
            fbeta.append(fbeta_score(y[train_index],prdict_y,beta=2))
            T.append(end-start)
    else:
        for train_index,test_index in cv.split(X, y):
    
            start=time.time()
            pipeline.fit(X[test_index],y[test_index])
            end=time.time()
            prdict_y=pipeline.predict(X[train_index])
            prob_y=pipeline.predict_proba(X[train_index])
            rec.append(recall_score(y[train_index],prdict_y))
            G_m.append(np.sqrt(recall_score(y[train_index],prdict_y)*recall_score(y[train_index],prdict_y,pos_label=0)))
            pre.append(precision_score(y[train_index],prdict_y))
            bacc.append(balanced_accuracy_score(y[train_index],prdict_y))
            fbeta.append(fbeta_score(y[train_index],prdict_y,beta=2))
            T.append(end-start)



    return G_m,bacc,fbeta,rec,pre,T


In [36]:
from sklearn.ensemble import BaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
models, names = get_models()
i=2
result = fivetrials(X_sample,y_sample,models[2],sample=RandomUnderSampler(random_state=0))

G_mean=result[0]
Bacc=result[1]
# summarize performance
recall=result[3]
Fbeta=result[2]
T=result[-1]
acc_s=np.mean(np.array(result)[:4,:],axis=0)
# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % (names[2],np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % (names[2],np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % (names[2],np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % (names[i],np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % (names[i],np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % (names[i],np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % (names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>BAG: Average G-mean:0.812(0.006) 
>BAG: Average Balanced_Acc: 0.813(0.005) 
>BAG: Average Fbeta: 0.690(0.008)
>BAG: Average Recall: 0.769(0.021)
>BAG: Average Training time: 3.824(0.040)
>BAG: Average accuracy_score: 0.771(0.010)
>BAG: Average Score: 0.766(0.010)


In [313]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import fbeta_score
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import BaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
models, names = get_models()
# from sklearn.neural_network import MLPClassifier
# evaluate each model
for i in range(len(models)):
	# evaluate the model and store results
	result = fivetrials(X_sample,y_sample,models[i])

	G_mean=result[0]
	Bacc=result[1]
# summarize performance
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	# summarize and store
	print('>%s: Average G-mean:%.3f(%.3f) ' % (names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % (names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % (names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % (names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % (names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % (names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % (names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>LR: Average G-mean:0.824(0.006) 
>LR: Average Balanced_Acc: 0.834(0.005) 
>LR: Average Fbeta: 0.711(0.011)
>LR: Average Recall: 0.705(0.012)
>LR: Average Training time: 0.553(0.171)
>LR: Average accuracy_score: 0.768(0.009)
>LR: Average Score: 0.768(0.009)
>SVM: Average G-mean:0.716(0.010) 
>SVM: Average Balanced_Acc: 0.753(0.007) 
>SVM: Average Fbeta: 0.543(0.014)
>SVM: Average Recall: 0.519(0.015)
>SVM: Average Training time: 20.555(1.283)
>SVM: Average accuracy_score: 0.633(0.012)
>SVM: Average Score: 0.627(0.011)
>BAG: Average G-mean:0.674(0.012) 
>BAG: Average Balanced_Acc: 0.723(0.008) 
>BAG: Average Fbeta: 0.486(0.016)
>BAG: Average Recall: 0.464(0.016)
>BAG: Average Training time: 3039.655(427.249)
>BAG: Average accuracy_score: 0.587(0.013)
>BAG: Average Score: 0.581(0.013)
>Adaboost: Average G-mean:0.757(0.009) 
>Adaboost: Average Balanced_Acc: 0.778(0.007) 
>Adaboost: Average Fbeta: 0.610(0.013)
>Adaboost: Average Recall: 0.600(0.014)
>Adaboost: Average Training time: 13.291

In [314]:
models, names = get_models()
# from sklearn.neural_network import MLPClassifier
# evaluate each model
for i in range(len(models)):
	# evaluate the model and store results
	result = fivetrials(X_sample,y_sample,models[i],sample=SMOTE(random_state=0))

	G_mean=result[0]
	Bacc=result[1]
# summarize performance
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	# summarize and store
	print('>%s: Average G-mean:%.3f(%.3f) ' % ('SMOTE'+names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('SMOTE'+names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>SMOTELR: Average G-mean:0.852(0.005) 
>SMOTELR: Average Balanced_Acc: 0.857(0.005) 
>SMOTELR: Average Fbeta: 0.756(0.008)
>SMOTELR: Average Recall: 0.766(0.011)
>SMOTELR: Average Training time: 0.734(0.071)
>SMOTELR: Average accuracy_score: 0.808(0.007)
>SMOTELR: Average Score: 0.806(0.007)
>SMOTESVM: Average G-mean:0.870(0.007) 
>SMOTESVM: Average Balanced_Acc: 0.872(0.006) 
>SMOTESVM: Average Fbeta: 0.781(0.010)
>SMOTESVM: Average Recall: 0.815(0.015)
>SMOTESVM: Average Training time: 52.789(1.931)
>SMOTESVM: Average accuracy_score: 0.835(0.010)
>SMOTESVM: Average Score: 0.826(0.009)
>SMOTEBAG: Average G-mean:0.750(0.010) 
>SMOTEBAG: Average Balanced_Acc: 0.773(0.008) 
>SMOTEBAG: Average Fbeta: 0.609(0.015)
>SMOTEBAG: Average Recall: 0.585(0.015)
>SMOTEBAG: Average Training time: 4465.234(360.818)
>SMOTEBAG: Average accuracy_score: 0.679(0.012)
>SMOTEBAG: Average Score: 0.672(0.012)
>SMOTEAdaboost: Average G-mean:0.784(0.012) 
>SMOTEAdaboost: Average Balanced_Acc: 0.795(0.009) 
>SMO

In [76]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import fbeta_score
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import BaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
models, names = get_models()
# from sklearn.neural_network import MLPClassifier
# evaluate each model
for i in range(len(models)):
	# evaluate the model and store results
	result = fivetrials(X_sample,y_sample,models[i],sample=RandomUnderSampler(random_state=0))

	G_mean=result[0]
	Bacc=result[1]
# summarize performance
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	# summarize and store
	print('>%s: Average G-mean:%.3f(%.3f) ' % ('RUS'+names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('RUS'+names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % ('RUS'+names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % ('RUS'+names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % ('RUS'+names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % ('RUS'+names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % ('RUS'+names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>RUSLR: Average G-mean:0.877(0.003) 
>RUSLR: Average Balanced_Acc: 0.877(0.003) 
>RUSLR: Average Fbeta: 0.787(0.004)
>RUSLR: Average Recall: 0.874(0.011)
>RUSLR: Average Training time: 0.174(0.019)
>RUSLR: Average accuracy_score: 0.854(0.005)
>RUSLR: Average Score: 0.854(0.005)
>RUSSVM: Average G-mean:0.867(0.003) 
>RUSSVM: Average Balanced_Acc: 0.867(0.003) 
>RUSSVM: Average Fbeta: 0.771(0.004)
>RUSSVM: Average Recall: 0.865(0.015)
>RUSSVM: Average Training time: 2.784(0.114)
>RUSSVM: Average accuracy_score: 0.843(0.005)
>RUSSVM: Average Score: 0.837(0.005)
>RUSBAG: Average G-mean:0.853(0.004) 
>RUSBAG: Average Balanced_Acc: 0.853(0.004) 
>RUSBAG: Average Fbeta: 0.751(0.005)
>RUSBAG: Average Recall: 0.850(0.014)
>RUSBAG: Average Training time: 333.771(11.679)
>RUSBAG: Average accuracy_score: 0.827(0.007)
>RUSBAG: Average Score: 0.819(0.007)
>RUSAdaboost: Average G-mean:0.838(0.005) 
>RUSAdaboost: Average Balanced_Acc: 0.838(0.005) 
>RUSAdaboost: Average Fbeta: 0.728(0.008)
>RUSAdaboos

# DRM(P)   
Mathod in ”Discriminative Ridge Machine: A Classifier for High-Dimensional Data or Imbalanced Data“

In [12]:
class Poly_close_solution:
    def __init__(self,kernel='self._poly',penalty = None,gamma=1,degree = 2,coef0=0,alpha=1,beta=1):
        self.W = None
        self.penalty = penalty
        self.d = degree
        self.b = coef0
        self.alpha=alpha
        self.beta=beta
        self.g=gamma
        self.kernel=kernel
        
    def _poly(self,X_1,X_2):
        return (self.g*(X_1.dot(X_2.T))+self.b)**self.d
    def _rbf(self,x,y):
        return np.exp(-self.g*np.sum((x[...,None,:]-y)**2,axis=2))

    def poly_B_matrix(self,X,Y):
        n=Y[Y==1].shape[0]
        X_sample_split=np.array(np.split(X,np.array([X.shape[0]-n])))
        B=np.zeros((X.shape[0],X.shape[0]))
        I=0
        for  m in X_sample_split:
            I+=m.shape[0]
            B[I-m.shape[0]:I,I-m.shape[0]:I]=eval(self.kernel+'(m,m)')/m.shape[0]
        return B
    
    def K_x(self,X,x_t):
        return eval(self.kernel+'(X,x_t)')
    
    def QplusBeta(self,X,Y):
        s=time.time()
        K=eval(self.kernel+'(X,X)')
        H_p=np.diag(np.diagonal(K))
        B_p=self.poly_B_matrix(X,Y)
        Qplus_beta=K+self.alpha*(H_p-B_p)+np.diag([self.beta]*X.shape[0])
        e=time.time()
        return K,Qplus_beta
    def fit(self,invQ,X,x_t):
        
        
        W=invQ.dot(self.K_x(X,x_t))##求逆耗时
        
        return W.reshape(-1)
    def delta_phi(self,K,invQ,X,Y,x_t,j):
        
        w=self.fit(invQ,X,x_t)
        w_noty=w

        w_y=np.zeros((X.shape[0]))
        indice=np.argwhere(Y==j).reshape(-1)
        w_noty[indice]=0
        w_y[indice]=w[indice]
        delta=np.dot(w_y,K.dot(w_y))+np.dot(w_noty,K.dot(w_noty))-2*np.dot(w_y,self.K_x(X,x_t))
        return delta
    def predict(self,K,invQ,X,Y,x_t):
        delta=[]
        for k in np.unique(Y):
            delta.append(self.delta_phi(K,invQ,X,Y,x_t,k))
        return np.argmin(delta)


In [13]:
'''Return the average performance on CV sets for DRM'''
from sklearn.metrics import fbeta_score
from numpy.linalg import inv,det
metrics_names=['G-mean','Balanced_acc','F1_score','Recall','Precision']
def DRM_FiveCV(X,y,model=Poly_close_solution(
                                )):
    
    G_m,bacc,f2,rec,pre=list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                K,Q_beta=model.QplusBeta(X[train_index],y[train_index])
                invQ=inv(Q_beta)
                y_test_label=list()
                for i in range(y[test_index].shape[0]):
                    y_test_label.append(model.predict(K,invQ,X[train_index],y[train_index],X[test_index][i]))

                rec.append(recall_score(y[test_index],y_test_label))
                pre.append(precision_score(y[test_index],y_test_label))
                G_m.append(math.sqrt(recall_score(y[test_index],y_test_label)*recall_score(y[test_index],y_test_label,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],y_test_label))
                f2.append(fbeta_score(y[test_index],y_test_label,beta=2))
 
    return G_m,bacc,f2,rec,pre


In [317]:
'''Parameter selection for DRM'''
X_tr_ord=np.concatenate((x_tr[y_tr==0],x_tr[y_tr==1]))
y_tr_ord=np.concatenate((y_tr[y_tr==0],y_tr[y_tr==1]))


degree=np.array([2,3,4,5,8,10])
gamma=np.array([0.001,0.01,0.1,1,10])
parameter=np.array([0.01,0.1,1,10,100])
all_results_Ldrm = []
import time
import math
# for k in mean_lr:
start=time.time()

for k in parameter:
    
    for d in degree:
        Lo_score=np.mean(DRM_FiveCV(X_tr_ord,y_tr_ord,model=Poly_close_solution(alpha=k,degree=d
                            )),axis=1)

        metric_res = {'alpha':k,'degree':d}

        for name, value in zip(metrics_names, Lo_score):
    #             print(name, ': ', value)
                metric_res[name] = value


        all_results_Ldrm.append(metric_res)
end=time.time()
print("DRM_L_fivefold-cv time:%.2f"%(end-start))

DRM_L_fivefold-cv time:6844.50


In [318]:
pd.DataFrame(all_results_Ldrm).to_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/code/0514CV_select_DRML_celebBangs.csv')


In [14]:
'''Return the average performance on five trials '''
def fivetrials_DRM(X,y):
    
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    
    for train_index,test_index in cv.split(X, y):

        start=time.time()
        P_model=Poly_close_solution(alpha=0.1,degree=2)
        K,Q_beta=P_model.QplusBeta(X[test_index],y[test_index])
        invQ=inv(Q_beta)
        end=time.time()
        prdict_y=[]
        for i in range(y[train_index].shape[0]):
            prdict_y.append(P_model.predict(K,invQ,X[test_index],y[test_index],X[train_index][i]))
        
        
        print(classification_report(y[train_index],prdict_y))
        rec.append(recall_score(y[train_index],prdict_y))
        G_m.append(np.sqrt(recall_score(y[train_index],prdict_y)*recall_score(y[train_index],prdict_y,pos_label=0)))
        pre.append(precision_score(y[train_index],prdict_y))
        bacc.append(balanced_accuracy_score(y[train_index],prdict_y))
        fbeta.append(fbeta_score(y[train_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
        T.append(end-start)


    return G_m,bacc,fbeta,rec,pre,T

In [15]:

'''Output the training time of DRM'''
def fivetrials_DRM_(X,y):
    
    
    T=list()
    for train_index,test_index in cv.split(X, y):

        start=time.time()
        P_model=Poly_close_solution(alpha=0.1,degree=2)
        K,Q_beta=P_model.QplusBeta(X[test_index],y[test_index])
        invQ=inv(Q_beta)
        end=time.time()

        T.append(end-start)

    return T

In [322]:
X_sample_ord=np.concatenate((X_sample[y_sample==0],X_sample[y_sample==1]))
y_sample_ord=np.concatenate((y_sample[y_sample==0],y_sample[y_sample==1]))
result_DRM = fivetrials_DRM(X_sample_ord,y_sample_ord)

G_mean=result_DRM[0]
Bacc=result_DRM[1]
# summarize performance
recall=result_DRM[3]
Fbeta=result_DRM[2]
T=result_DRM[-1]
acc_s=np.mean(np.array(result_DRM)[:4,:],axis=0)
	# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % ('DRM:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('DRM:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('DRM:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('DRM:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('DRM:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('DRM:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('DRM:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     11012
         1.0       0.84      0.68      0.75      1953

    accuracy                           0.93     12965
   macro avg       0.89      0.83      0.85     12965
weighted avg       0.93      0.93      0.93     12965

              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96     11013
         1.0       0.82      0.73      0.77      1952

    accuracy                           0.93     12965
   macro avg       0.89      0.85      0.87     12965
weighted avg       0.93      0.93      0.93     12965

              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96     11013
         1.0       0.84      0.69      0.76      1953

    accuracy                           0.93     12966
   macro avg       0.89      0.84      0.86     12966
weighted avg       0.93      0.93      0.93     12966

              preci

In [17]:
X_sample_ord=np.concatenate((X_sample[y_sample==0],X_sample[y_sample==1]))
y_sample_ord=np.concatenate((y_sample[y_sample==0],y_sample[y_sample==1]))
T=fivetrials_DRM_(X_sample_ord,y_sample_ord)
print('>%s: Average Training time: %.3f(%.3f)' % ('DRM:',np.mean(T),np.std(T)))

>DRM:: Average Training time: 1.645(0.145)


# Max-mean loss method
" Q. Xu, X. M. Xuan, Nonlinear regression without iid assumption"

In [81]:
'''Data preprocessing for max-mean loss method'''

def split(x,y):
    k=int(y.shape[0]/sum(y))-1
    n=y[y==1].shape[0]
    m=int((x.shape[0]-n)/k)
    indice=[i*m for i in range(1,k)]
    indice.append(x.shape[0]-n)
    x_c=np.concatenate((x[y==0],x[y==1]))
    y_c=np.concatenate((np.array([0]*(x.shape[0]-n)),np.array([1]*n)))
    return np.array(np.split(x_c,indice)),np.array(np.split(y_c,indice))

In [77]:
''' Class solution_XX() is the algorithm based on Xiaohua,Xuan[2019], the improvement is that
we add a parameter of "penalty".'''
class solution_XX:
    
    
    def __init__(self,penalty = None,Lambda = 0.03,a = 0.5,epochs = 200):
        self.W = None
        self.penalty = penalty
        self.Lambda = Lambda
        self.a = a
        self.epochs =epochs
        self.sigmoid = lambda x:1/(1 + np.exp(-x))


    def f_XX(self,X,Y):
        if self.penalty=='l1':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(np.abs(self.W)) for x,y in zip(X,Y)])
        elif self.penalty=='l2':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(self.W**2) for x,y in zip(X,Y)])#pre_Xtrain,pre_Ytrain
        else:f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 ) for x,y in zip(X,Y)])

        return f         
    def Gf_XX(self,X,Y):#To compute the Derivative matrix, the shape of which is N*2
        if self.penalty=='l1':d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+self.Lambda*np.sign(self.W )for x,y in zip(X,Y)])
        elif self.penalty=='l2':
            d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+2*self.Lambda*self.W for x,y in zip(X,Y)])

        else:d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W)))) for x,y in zip(X,Y)])
        return d
    def direction_XX(self,X,Y):
        gra=self.Gf_XX(X,Y)
        p=matrix(gra.dot(gra.T),tc='d')
        q=matrix(-self.f_XX(X,Y),tc='d')
        G=matrix(np.diag(np.array([-1]*(Y.shape[0]))),tc='d')#N=20
        h=matrix(np.array([[0]]*(Y.shape[0])),tc='d')
        A=matrix([[1.0]]*(Y.shape[0]))
        b=matrix([1.0])
        solvers.options['show_progress'] = False
        sol = solvers.qp(p,q,G,h,A,b)
        t=np.array(sol['x'])
        d= -(gra.T.dot(t))
        return d.reshape((X_train.shape[-1],))

    def fit(self,X,Y):
        
        call=[]
        pre=[]
        loss=[]
        testloss=[]
        np.random.seed(1324)
        self.W=np.random.random((X_train.shape[-1],))*2-1
        n=y_te[y_te==1.].shape[0]
        for k in range(200):
            d=self.direction_XX(X,Y)
            if np.linalg.norm(d)//10**(-7) < 25:
                break
            sigma=0.8
            f_1=np.max(self.f_XX(X,Y))
            w=self.W
            self.W=d*sigma+w
            while np.max(self.f_XX(X,Y))>np.max(f_1):
                sigma=sigma*0.8
                self.W=d*sigma+w
            self.W=d*sigma+w

        return self.W

In [132]:
'''Return the performances of max-mean loss regression on five random trials'''

def fivetrials_XX(X,y,model=solution_XX()):
    pipeline = model
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    
    for train_index,test_index in cv.split(X, y):
        
        start=time.time()

        pre_X,pre_Y=split(X[test_index],y[test_index])

        w_x=pipeline.fit(pre_X,pre_Y)
        end=time.time()
        prdict_y=pipeline.sigmoid(X[train_index].dot(w_x))
        prdict_y[prdict_y>=0.5]=1
        prdict_y[prdict_y<0.5]=0

        print(classification_report(y[train_index],prdict_y))
        
        rec.append(recall_score(y[train_index],prdict_y))
        G_m.append(np.sqrt(recall_score(y[train_index],prdict_y)*recall_score(y[train_index],prdict_y,pos_label=0)))
        pre.append(precision_score(y[train_index],prdict_y))
        bacc.append(balanced_accuracy_score(y[train_index],prdict_y))
        fbeta.append(fbeta_score(y[train_index],prdict_y,beta=2))
        T.append(end-start)


    return G_m,bacc,fbeta,rec,pre,T

In [133]:
result_XX = fivetrials_XX(X_sample_XX,y_sample)

G_mean=result_XX[0]
Bacc=result_XX[1]
# summarize performance
recall=result_XX[3]
Fbeta=result_XX[2]
T=result_XX[-1]
acc_s=np.mean(np.array(result_XX)[:4,:],axis=0)
	# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % ('max_mean loss:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('max_mean loss:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('max_mean loss:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('max_mean loss:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('max_mean loss:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('max_mean loss:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('max_mean loss:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          


              precision    recall  f1-score   support

         0.0       0.96      0.91      0.93     11012
         1.0       0.61      0.77      0.68      1953

    accuracy                           0.89     12965
   macro avg       0.78      0.84      0.81     12965
weighted avg       0.90      0.89      0.90     12965

              precision    recall  f1-score   support

         0.0       0.97      0.88      0.92     11013
         1.0       0.56      0.84      0.67      1952

    accuracy                           0.88     12965
   macro avg       0.76      0.86      0.80     12965
weighted avg       0.91      0.88      0.89     12965

              precision    recall  f1-score   support

         0.0       0.96      0.90      0.93     11013
         1.0       0.60      0.80      0.68      1953

    accuracy                           0.89     12966
   macro avg       0.78      0.85      0.81     12966
weighted avg       0.91      0.89      0.89     12966

              preci

## Mean_uncertain method

In [38]:
def sigmoid_(x,u):
    return 1.0/(1.0+np.exp(-x-u))

In [43]:
def meanuncertainty(x,n):
    r=[]
    for i in range(0,len(x)+1-n,n//10):
        r.append(np.mean(x[i:i+n]))
    return min(r),max(r)

In [39]:

'''Return the worst-case trainin error'''
def equa_m(x,n,r):
    pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
    ini_err=y_tr-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
    
        if y_tr[k]==1:
            ini_err[k]=r*ini_err[k]

    
    return meanuncertainty(ini_err,n)[1]
def equa_fivem(x,n,r,Mod,x_train,y_train):
    pre_in=(np.dot(x_train,Mod.coef_.T)+Mod.intercept_).reshape(x_train.shape[0],)
    
    ini_err=y_train-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
        if y_train[k]==1:
            ini_err[k]=r*ini_err[k]

    
    return meanuncertainty(ini_err,n)[1]

In [49]:
'''Estimate upper mean'''
mean_lr=np.array([fsolve(lambda x:equa_m(x,n,0.5*y_sample.shape[0]/sum(y_sample)),0.5 )
                  for n in range(20,500,10)])

In [294]:
'''Estimate volatility uncertainty'''
var_lr=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] )
                  for n in range(50,1000,10)])

In [310]:
pd.DataFrame(var_lr).to_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/code/0512_lrvar_mean_celebbangs.csv')


In [228]:
def select(arr,n,k):##arr为1-D
    per=[]
    A=abs(arr-k)

    for j in np.sort(A)[:n]:
        per.append(list(A).index(j))
    return arr[per]

## Choice optimal upper mean on  Cross-validation set

In [51]:

"""We use 'evaluate_model' to obtain the average performance in 5-fold cross-validation set, for fixed upper mean 'm' """

metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                model.fit(X[train_index],y[train_index])
                prob_y=sigmoid_((np.dot(X[test_index],model.coef_.T)+model.intercept_),m)
                
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,rec,pre
    
           
               

In [52]:
'''We conduct fivefold cv to select optimal window size N'''

all_results = []
c=2
# for k in mean_lr:
strat=time.time()
for k in mean_lr.reshape(-1):
    result=np.mean(evaluate_model(x_tr,y_tr,k),axis=1)
    metric_res = {'window':10*c,'upper_mean': k}
    c+=1
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_results.append(metric_res)

In [53]:
eva=pd.DataFrame(all_results)
bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]
N_optimal=bias['window'].values[0]
print("optimal window size N by fivefold CV: %.3f "%(bias['window']))


optimal window size N by fivefold CV: 30.000 


In [54]:
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def Fivetrails_mean(k,X,y,model=LogisticRegression()):
    mean=mean_lr.reshape(-1)
    G_m,bacc,f2,rec,pre,T=list(),list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                s_1=time.time()

                model.fit(X[test_index],y[test_index])
                '''mean_u is the upper mean with optimal window size 30'''
                mean_u=fsolve(lambda x:equa_fivem(x,N_optimal,0.5*(k+1),model,X[test_index],y[test_index]),0.5 )
                t_1=time.time()

                print("The upper_mean : %.3f "%(mean_u))
                prob_y=sigmoid_((np.dot(X[train_index],model.coef_.T)+model.intercept_).reshape(X[train_index].shape[0],),mean_u
                             )
                
                
                
                prdict_y=np.round(prob_y)
                print(classification_report(y[train_index],prdict_y))
                rec.append(recall_score(y[train_index],prdict_y))
                pre.append(precision_score(y[train_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[train_index],prdict_y)*recall_score(y[train_index],prdict_y,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[train_index],prdict_y))
                f2.append(fbeta_score(y[train_index],prdict_y,beta=max(2,np.log(k))))
                T.append(t_1-s_1)
    return G_m,bacc,f2,rec,pre,T

In [55]:
result = Fivetrails_mean(y_sample.shape[0]/sum(y_sample)-1,X_sample,y_sample)
# 
G_mean=result[0]
Bacc=result[1]
# summarize performance
recall=result[3]
Fbeta=result[2]
T=result[-1]
acc_s=np.mean(np.array(result)[:4,:],axis=0)

print('>%s: Average G-mean:%.3f(%.3f) ' % ('LR_mean:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('LR_mean:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('LR_mean:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('LR_mean:',np.mean(recall),np.std(recall)))
print('>%s: Average Training time: %.3f(%.3f)' % ('LR_mean:',np.mean(T),np.std(T)))
print('>%s: Average Score: %.3f(%.3f)' % ('LR_mean:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))


The upper_mean : 3.040 
              precision    recall  f1-score   support

         0.0       0.98      0.86      0.92     11012
         1.0       0.53      0.89      0.67      1953

    accuracy                           0.87     12965
   macro avg       0.75      0.87      0.79     12965
weighted avg       0.91      0.87      0.88     12965

The upper_mean : 2.751 
              precision    recall  f1-score   support

         0.0       0.98      0.87      0.92     11013
         1.0       0.55      0.91      0.68      1952

    accuracy                           0.87     12965
   macro avg       0.76      0.89      0.80     12965
weighted avg       0.92      0.87      0.88     12965

The upper_mean : 2.726 
              precision    recall  f1-score   support

         0.0       0.98      0.88      0.93     11013
         1.0       0.57      0.89      0.69      1953

    accuracy                           0.88     12966
   macro avg       0.77      0.89      0.81     12966
we

## Volatility-uncertain method


In [25]:
'''F_u and F_L compute the maximal and minimal probability of Y=1 '''
 
def F_u(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[1]*st.norm.cdf(c/x[1])/(x[0]+x[1]))
        else:
            p.append(1-2*x[0]*st.norm.cdf(-c/x[0])/(x[0]+x[1]))
    return np.array(p)
def F_L(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[0]*st.norm.cdf(c/x[0])/(x[0]+x[1]))
        else:
            p.append(1-2*x[1]*st.norm.cdf(-c/x[1])/(x[0]+x[1]))
    return np.array(p)

In [285]:
'''equa_v returns the maximal and minimal err obtained by LR for fixed window size n'''

def equa_v(x,n):
 
    pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
    for k in range(y_tr.shape[0]):
    
        if y_tr[k]==1:
            err_u[k]=3.4*err_u[k]
            err_L[k]=3.4*err_L[k]
    
    return np.array([meanuncertainty(err_u,n)[1],meanuncertainty(err_L,n)[0]])

## Choice optimal window size on  Cross-validation set

In [22]:
def var_evaluate_model(X,y,m,model=LogisticRegression()):
    
    G_m,bacc,f2,rec,pre=list(),list(),list(),list(),list()
   
    for train_index,test_index in cv.split(X,y):
                model.fit(X[train_index],y[train_index])
                prob_y=F_u(m,(np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],))
        
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))

                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return G_m,bacc,f2,rec,pre

In [29]:
def Fivetrails_var(X,y,model=LogisticRegression()):
    
    G_m,bacc,f2,rec,pre=list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                model.fit(X[test_index],y[test_index])
                all_results = []
                c=0

                for i in var_lr[:10]:
                    result=np.mean(var_evaluate_model(X[train_index],y[train_index],i),axis=1)
                    metric_res = {'window':10+5*c,'upper_mean': i}
                    c+=1
                    for name, value in zip(metrics_names, result):
                            metric_res[name] = value


                    all_results.append(metric_res)
                eva=pd.DataFrame(all_results)
                bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]##select the optimal window size
                print(np.array(bias['upper_mean']))##the optimal lower and upper volatilities
                
                prob_y=F_u(np.array(bias['upper_mean'])[0],(np.dot(X[train_index],model.coef_.T)+model.intercept_).reshape(X[train_index].shape[0],))
                
                
                prdict_y=np.round(prob_y)
                print(classification_report(y[train_index],prdict_y))
                rec.append(recall_score(y[train_index],prdict_y))
                pre.append(precision_score(y[train_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[train_index],prdict_y)*recall_score(y[train_index],prdict_y,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[train_index],prdict_y))
                f2.append(fbeta_score(y[train_index],prdict_y,beta=2))
 
    return G_m,bacc,f2,rec,pre

In [33]:
result = Fivetrails_var(X_sample,y_sample)
# 
G_mean=result[0]
Bacc=result[1]
# summarize performance
recall=result[3]
Fbeta=result[2]
T=result[-1]
acc_s=np.mean(np.array(result)[:4,:],axis=0)

print('>%s: Average G-mean:%.3f(%.3f) ' % ('LR_volatility:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('LR_volatility:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('LR_volatility:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('LR_volatility:',np.mean(recall),np.std(recall)))

print('>%s: Average Training time: %.3f(%.3f)' % ('LR_mean:',np.mean(T),np.std(T)))
print('>%s: Average Score: %.3f(%.3f)' % ('LR_mean:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))


[array([0.08194467, 2.80559692])]
              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94     11012
         1.0       0.63      0.82      0.71      1953

    accuracy                           0.90     12965
   macro avg       0.80      0.87      0.83     12965
weighted avg       0.92      0.90      0.90     12965

[array([0.08194467, 2.80559692])]
              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94     11013
         1.0       0.63      0.87      0.73      1952

    accuracy                           0.90     12965
   macro avg       0.80      0.89      0.84     12965
weighted avg       0.92      0.90      0.91     12965

[array([0.02595335, 2.68712139])]
              precision    recall  f1-score   support

         0.0       0.97      0.92      0.94     11013
         1.0       0.65      0.84      0.73      1953

    accuracy                           0.91     12966
   macro avg       0.81   